## Overview
    Pyro is a Probabilistic Programming.
    Unlike tensorflow for example, Pyro often uses nondeterministic distribution to illustrate problems and solve them.

In [1]:
# import some dependencies
import torch
from torch.autograd import Variable

import pyro
import pyro.distributions as dist

### stochastic function
    We will often call stochastic functions models, since stochastic functions can be used to represent simplified or abstract descriptions of a process by which data are generated. 

### Primitive Stochastic Functions
    For example, to draw a sample x from the unit normal distribution N(0,1) we do the following:

In [3]:
mu = Variable(torch.zeros(1))   # mean zero
sigma = Variable(torch.ones(1)) # unit variance
x = dist.normal(mu, sigma)      # x is a sample from N(0,1)
print(x)

Variable containing:
 0.4495
[torch.FloatTensor of size 1]



    Note that the parameters passed to dist.normal are PyTorch Variables. This is necessary because we want to make use of PyTorch’s fast tensor math and autograd capabilities. 

In [6]:
log_p_x = dist.normal.log_pdf(x, mu, sigma)
print(log_p_x)

Variable containing:
-1.0200
[torch.FloatTensor of size 1]



### The pyro.sample Primitive
    Just like a direct call to dist.normal, this returns a sample from the unit normal distribution. The crucial difference is that this sample is named

In [9]:
x = pyro.sample("my_sample", dist.normal, mu, sigma)
print(x)
# fn is an arbitrary stochastic function
# y = pyro.sample("my_sample", fn, arg1, arg2)
# print(y)

Variable containing:
 0.6675
[torch.FloatTensor of size 1]



### A Simple Model
    Let’s suppose we have a bunch of data with daily mean temperatures and cloud cover. We want to reason about how temperature interacts with whether it was sunny or cloudy. A simple stochastic function that does that is given by:

In [15]:
def weather():
    # return 0 or 1
    cloudy = pyro.sample('cloudy', dist.bernoulli,
                         Variable(torch.Tensor([0.3])))
    cloudy = 'cloudy' if cloudy.data[0] == 1.0 else 'sunny'
    mean_temp = {'cloudy': [55.0], 'sunny': [75.0]}[cloudy]
    sigma_temp = {'cloudy': [10.0], 'sunny': [15.0]}[cloudy]
    #  joint probability distribution
    temp = pyro.sample('temp', dist.normal,
                       Variable(torch.Tensor(mean_temp)),
                       Variable(torch.Tensor(sigma_temp)))
    return cloudy, temp.data[0]

for _ in range(3):
    print(weather())

('cloudy', 49.88711929321289)
('cloudy', 63.97058868408203)
('sunny', 88.5665283203125)


    Simulate a easy problem in the life.

In [20]:
def ice_cream_sales():
    cloudy, temp = weather()
    expected_sales = [200] if cloudy == 'sunny' and temp > 80.0 else [50]
    ice_cream = pyro.sample('ice_cream', dist.normal,
                            Variable(torch.Tensor(expected_sales)),
                            Variable(torch.Tensor([10.0])))
    return ice_cream
print(ice_cream_sales())

Variable containing:
 206.7471
[torch.FloatTensor of size 1]



### Universality: Stochastic Recursion, Higher-order Stochastic Functions, and Random Control Flow
    For example we can define a geometric distribution like so:

In [24]:
def geometric(p, t=None):
    if t is None:
        t = 0
    x = pyro.sample("x_{}".format(t), dist.bernoulli, p)
    if torch.equal(x.data, torch.zeros(1)):
        return x
    else:
        return x + geometric(p, t+1)

print(geometric(Variable(torch.Tensor([0.5]))))

Variable containing:
 0
[torch.FloatTensor of size 1]



    We are also free to define stochastic functions that accept as input or produce as output other stochastic functions:
    (The code below is relatively hard to understand)

In [29]:
def normal_product(mu, sigma):
    z1 = pyro.sample("z1", dist.normal, mu, sigma)
    z2 = pyro.sample("z2", dist.normal, mu, sigma)
    y = z1 * z2
    return y

def make_normal_normal():
    mu_latent = pyro.sample("mu_latent", dist.normal,
                            Variable(torch.zeros(1)),
                            Variable(torch.ones(1)))
    # It seems that torch.ones(1) is sigma here ?
    fn = lambda sigma: normal_product(mu_latent, sigma)
    return fn

print(make_normal_normal()(Variable(torch.ones(1))))

Variable containing:
 0.9202
[torch.FloatTensor of size 1]

